<a href="https://colab.research.google.com/github/stampejp/data-mining/blob/master/homework/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📝 Homework 2: Data Wrangling - From Business Question to Analysis

<a href="https://colab.research.google.com/github/bradleyboehmke/uc-bana-4080/blob/main/homework/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## 📂 Instructions

This homework is based on the Lab we worked through in Thursday's class.  So, if you completed that Lab, you can use that notebook for the homework.

Complete the tasks below in this Jupyter notebook. Most tasks require you to write Python code and use the output to answer **a separate online quiz**.

At the end, you’ll also upload this completed `.ipynb` notebook

---

In this homework, we’ll use **three datasets** from the Complete Journey retail grocery data:

1. **transactions** – product purchases by households (receipt-level detail)  
2. **demographics** – household-level demographic data  
3. **products** – metadata about products purchased  

This homework reinforces this week’s readings:

- **[Reading 10: Manipulating Data](https://bradleyboehmke.github.io/uc-bana-4080/10-manipulating-data.html)**
- **[Reading 11: Summarizing Data](https://bradleyboehmke.github.io/uc-bana-4080/11_aggregating_data.html)**
- **[Reading 12: Joining Data](https://bradleyboehmke.github.io/uc-bana-4080/12-joining-data.html)**

We will:
- Start with simple data exploration
- Progress to manipulating and summarizing data
- End with joining datasets to answer more complex questions
- Practice breaking business questions into **analytical steps**

You are encouraged to work in small groups of **2–4 students** but you must submit your own notebook.


## Setup

In [20]:
# If you don't have completejourney_py installed, run: pip install completejourney-py
from completejourney_py import get_data
import pandas as pd

# Load datasets
cj_data = get_data()
transactions = cj_data['transactions']
products = cj_data['products']
demographics = cj_data['demographics']

# Quick preview
transactions.head()


,household_id,store_id,basket_id,product_id,quantity,sales_value,retail_disc,coupon_disc,coupon_match_disc,week,transaction_timestamp
0,900,330,31198570044,1095275,1,0.50,0.00,0.0,0.0,1,2017-01-01 11:53:26
1,900,330,31198570047,9878513,1,0.99,0.10,0.0,0.0,1,2017-01-01 12:10:28
2,1228,406,31198655051,1041453,1,1.43,0.15,0.0,0.0,1,2017-01-01 12:26:30
3,906,319,31198705046,1020156,1,1.50,0.29,0.0,0.0,1,2017-01-01 12:30:27
4,906,319,31198705046,1053875,2,2.78,0.80,0.0,0.0,1,2017-01-01 12:30:27


## Part 1 – Basic Exploration


**Q0:** How many transactions are in our dataset, what is the date range, how many households have demographic data, how many products exist, and what are the min/max/mean sales values?  

**Step-by-step instructions:**
1. Use `.shape[0]` on `transactions` to count rows.  
2. Use `.min()` and `.max()` on `transaction_timestamp` to find the date range.  
3. Use `.shape[0]` on `demographics` and `products` to get counts.  
4. Use `.min()`, `.max()`, `.mean()` on `sales_value` for basic stats.


In [23]:
# Starter code with blanks to fill
# total number of transactions
num_transactions = transactions.shape[0]
num_transactions

1469307

In [24]:
# date range of transactions
min_date = transactions['transaction_timestamp'].min()
max_date = transactions['transaction_timestamp'].max()

min_date, max_date

(Timestamp('2017-01-01 11:53:26'), Timestamp('2018-01-01 04:01:20'))

In [25]:
# number of unique households and products
num_households = demographics.shape[0]
num_products = products.shape[0]

num_households, num_products

(801, 92331)

In [26]:
# summary statistics for sales_value
min_sales = transactions['sales_value'].min()
max_sales = transactions['sales_value'].max()
mean_sales = transactions['sales_value'].mean()

min_sales, max_sales, mean_sales

(0.0, 840.0, np.float64(3.12803218115751))


**Q1:** Which day had the highest total sales?  

**Step-by-step instructions:**
1. Create a new column `date` by extracting only the date from `transaction_timestamp` (`.dt.date`).  
2. Group by `date` and sum `sales_value`.  
3. Sort results in descending order.  
4. Select the top row.


In [27]:
transactions['date'] = transactions['transaction_timestamp'].dt.date
daily_sales = transactions.groupby('date')['sales_value'].sum()
highest_sales_day = daily_sales.sort_values(ascending=False).head(1)
display(highest_sales_day)

,sales_value
date,
2017-12-23,24994.47



**Q2:** What are the top 5 departments by total sales?  

**Step-by-step instructions:**
1. Join `transactions` to `products` on `product_id` using an inner join.  
2. Group by `department` and sum `sales_value`.  
3. Sort results in descending order.  
4. Display the top 5.


In [28]:
# Join transactions to products
transactions_products = pd.merge(transactions, products, on='product_id', how='inner')

# Group by department and sum sales_value
department_sales = transactions_products.groupby('department')['sales_value'].sum()

# Sort results in descending order and select the top 5
top_5_departments = department_sales.sort_values(ascending=False).head(5)

# Display the top 5
display(top_5_departments)

,sales_value
department,
GROCERY,2316393.89
DRUG GM,596827.45
FUEL,329594.45
PRODUCE,322858.82
MEAT,308575.33


## Part 2 – Manipulating Data


**Q3:** What is the average unit price for each department?  

**Step-by-step instructions:**
1. Create a `unit_price` column: `sales_value / quantity`.  
2. Join `transactions` to `products` to bring in `department`.  
3. Group by `department` and calculate the mean of `unit_price`.


In [29]:
# Create unit_price column
transactions['unit_price'] = transactions['sales_value'] / transactions['quantity']

# Join transactions to products
transactions_products_price = pd.merge(transactions, products, on='product_id', how='inner')

# Group by department and calculate mean unit_price
avg_unit_price_by_department = transactions_products_price.groupby('department')['unit_price'].mean()

# Display the average unit price by department
display(avg_unit_price_by_department)

,unit_price
department,
AUTOMOTIVE,7.216111
CHEF SHOPPE,2.522274
CNTRL/STORE SUP,3.150000
COSMETICS,4.138923
COUPON,1.296070
DELI,inf
DRUG GM,inf
ELECT &PLUMBING,1.000000
FLORAL,7.732635



**Q4:** Do we have missing values in `unit_price`?  

**Step-by-step instructions:**
1. Use `.isna().sum()` on `unit_price` to count missing values.  
2. Consider filtering rows where `quantity == 0` to see if that’s the cause.


In [30]:
# Check for missing values in unit_price
missing_unit_price = transactions['unit_price'].isna().sum()
print(f"Number of missing values in unit_price: {missing_unit_price}")

# Check if quantity == 0 is the cause
zero_quantity_missing_price = transactions[transactions['quantity'] == 0]['unit_price'].isna().sum()
print(f"Number of missing values in unit_price where quantity is 0: {zero_quantity_missing_price}")

Number of missing values in unit_price: 8820
Number of missing values in unit_price where quantity is 0: 8820


## Part 3 – Aggregations


**Q5:** Which income level spends the most on average?

*Hint:* Join transactions to demographics, group by income, calculate mean sales per income level.


In [31]:
# Join transactions to demographics
transactions_demographics = pd.merge(transactions, demographics, on='household_id', how='inner')

# Group by income and calculate mean sales_value
avg_sales_by_income = transactions_demographics.groupby('income')['sales_value'].mean()

# Sort results in descending order and display
highest_spending_income = avg_sales_by_income.sort_values(ascending=False)
display(highest_spending_income)

,sales_value
income,
175-199K,3.754513
250K+,3.724832
200-249K,3.703222
150-174K,3.541206
100-124K,3.481148
125-149K,3.457844
75-99K,3.327325
50-74K,3.149264
35-49K,2.978827



**Q6:** Do households with kids spend more (on average) than households without kids?  

*Hint:* Use `kid_count` to group households by creating a new column (e.g., `has_kids`) that identifies whether a household has kids (`kid_count > 0`) or not (`kid_count == 0`). Note that the tricky part of this step is that `kid_count` is not a numeric variable 🤔. Compute the average spend for those with kids and those without.


In [46]:
# Join transactions to demographics
transactions_demographics = pd.merge(transactions, demographics, on='household_id', how='inner')

# Create a 'has_kids' column
transactions_demographics['has_kids'] = transactions_demographics['kids_count'].apply(lambda x: 'With Kids' if x != 0 else 'Without Kids')

# Group by has_kids and calculate mean sales_value
avg_sales_by_kids = transactions_demographics.groupby('has_kids')['sales_value'].mean()

# Display the average sales by kid count for both groups
display(avg_sales_by_kids)

,sales_value
has_kids,
With Kids,3.167014



**Q7:** What are the top 5 departments by total quantity of items sold?  

*Hint:* Join to products, group by department, sum quantity, and sort.


In [33]:
# Join transactions to products
transactions_products_quantity = pd.merge(transactions, products, on='product_id', how='inner')

# Group by department and sum quantity
department_quantity = transactions_products_quantity.groupby('department')['quantity'].sum()

# Sort results in descending order and select the top 5
top_5_departments_by_quantity = department_quantity.sort_values(ascending=False).head(5)

# Display the top 5
display(top_5_departments_by_quantity)

,quantity
department,
FUEL,129662940
MISCELLANEOUS,21361882
GROCERY,1242944
DRUG GM,198635
PRODUCE,185444


## Part 4 – Joins for Deeper Insights


**Q8:** Which product is purchased most frequently?  

*Hint:* Group by `product_id`, sum quantity, then join to products for description.


In [34]:
# Group by product_id and sum quantity
product_quantity = transactions.groupby('product_id')['quantity'].sum()

# Find the product with the highest quantity and join to products for description
most_frequent_product = product_quantity.sort_values(ascending=False).head(1)

# Join with products dataframe to get product description
most_frequent_product_details = pd.merge(most_frequent_product, products, on='product_id', how='inner')

# Display the most frequently purchased product
display(most_frequent_product_details)

,product_id,quantity,manufacturer_id,department,brand,product_category,product_type,package_size
0,6534178,126868510,69,FUEL,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,None



**Q9:** Identify all products with “pizza” in `product_type` and find the one with the greatest total sales.  

*Hint:* Filter products where product_type contains "pizza" with `.str.contains("pizza", case=False, na=False)`, join to transactions, sum sales by product.


In [35]:
# Filter products for "pizza" in product_type
pizza_products = products[products['product_type'].str.contains("pizza", case=False, na=False)]

# Join with transactions
pizza_transactions = pd.merge(transactions, pizza_products, on='product_id', how='inner')

# Group by product and sum sales
pizza_sales = pizza_transactions.groupby(['product_id', 'product_type'])['sales_value'].sum()

# Find the pizza product with the greatest total sales
top_pizza_product = pizza_sales.sort_values(ascending=False).head(1)

# Display the result
display(top_pizza_product)

,,sales_value
product_id,product_type,
944139,PIZZA/TRADITIONAL,1344.5



**Q10:** Which product category brings in the most revenue for the highest-income households with kids?

*Hint:* Filter demographics for the highest income level & `kid_count > 0`, join to transactions and products, group by category and compute the sum of sales value.


In [36]:
# Filter demographics for the highest income level and kids_count > 0
highest_income_kids_households = demographics[(demographics['income'] == '250K+') | (demographics['income'] == '175-199K') | (demographics['income'] == '200-249K') & (demographics['kids_count'] != 0)]

# Join with transactions
highest_income_kids_transactions = pd.merge(transactions, highest_income_kids_households, on='household_id', how='inner')

# Join with products
highest_income_kids_products = pd.merge(highest_income_kids_transactions, products, on='product_id', how='inner')

# Group by product category and compute the sum of sales value
category_revenue = highest_income_kids_products.groupby('product_category')['sales_value'].sum()

# Find the product category with the highest revenue
top_category_highest_income_kids = category_revenue.sort_values(ascending=False).head(1)

# Display the result
display(top_category_highest_income_kids)

,sales_value
product_category,
COUPON/MISC ITEMS,8068.4



**Q11:** Which manufacturer has the highest total sales, and which department do they primarily sell in?  

*Hint:* Join transactions to products, group by manufacturer, sum sales, find top. Then, filter products for that top manufacturer and check which department(s) they are associated with.


In [37]:
# Join transactions to products
transactions_products_manufacturer = pd.merge(transactions, products, on='product_id', how='inner')

# Group by manufacturer and sum sales
manufacturer_sales = transactions_products_manufacturer.groupby('manufacturer_id')['sales_value'].sum()

# Find the manufacturer with the highest sales
top_manufacturer_id = manufacturer_sales.sort_values(ascending=False).head(1).index[0]
print(f"The manufacturer with the highest total sales is Manufacturer ID: {top_manufacturer_id}")

# Filter products for the top manufacturer and find their primary department
top_manufacturer_products = products[products['manufacturer_id'] == top_manufacturer_id]

# Count the occurrences of each department for the top manufacturer
department_counts = top_manufacturer_products['department'].value_counts()

# Get the primary department (the one with the highest count)
primary_department = department_counts.index[0]
print(f"They primarily sell in the department: {primary_department}")

The manufacturer with the highest total sales is Manufacturer ID: 69
They primarily sell in the department: GROCERY



**Q12:** For each income level, what is the most frequently purchased product category?  

*Hint:* Join demographics → transactions → products, group by income & category, count quantity, get top per income.


In [38]:
# Join demographics to transactions
demographics_transactions = pd.merge(demographics, transactions, on='household_id', how='inner')

# Join with products
demographics_transactions_products = pd.merge(demographics_transactions, products, on='product_id', how='inner')

# Group by income and product category, and count quantity
income_category_quantity = demographics_transactions_products.groupby(['income', 'product_category'])['quantity'].sum()

# Find the most frequently purchased product category for each income level
most_frequent_category_by_income = income_category_quantity.groupby('income').idxmax()

# Display the result
display(most_frequent_category_by_income)

,quantity
income,
100-124K,"(100-124K, COUPON/MISC ITEMS)"
125-149K,"(125-149K, COUPON/MISC ITEMS)"
15-24K,"(15-24K, COUPON/MISC ITEMS)"
150-174K,"(150-174K, COUPON/MISC ITEMS)"
175-199K,"(175-199K, COUPON/MISC ITEMS)"
200-249K,"(200-249K, COUPON/MISC ITEMS)"
25-34K,"(25-34K, COUPON/MISC ITEMS)"
250K+,"(250K+, COUPON/MISC ITEMS)"
35-49K,"(35-49K, COUPON/MISC ITEMS)"


## Homework Deliverable


- Implement the code to answer the above questions.  
- Once you have all your answers, go to the homework quiz on Canvas and submit your answers.  
- Save your notebook — you'll upload it on Canvas as part of the homework.
